## METHODE PLNE

In [1]:
from minilp import *
import numpy as np

In [2]:
lp = problem('PLNE')

#existence of a server i in a line j
e = np.array(lp.integer_var_list(10,0,1)).reshape(2,5)

#a server appears only one time
lp.add_constraint(sum(e.T[0]) <= 1)
lp.add_constraint(sum(e.T[1]) <= 1)
lp.add_constraint(sum(e.T[2]) <= 1)
lp.add_constraint(sum(e.T[3]) <= 1)
lp.add_constraint(sum(e.T[4]) <= 1)

#line j has limited space
s = [4,5]
z = [3,3,2,1,1]
for j in range(2):
    lp.add_constraint( sum(e[j]*z) <= s[j] )

lp.set_objective( sum(sum(e*[10,10,5,5,6])), sense=max)

In [3]:
res = lp.lp_solve()
print(res)

print("-------------")
for j in range(2):
    for i in range(4):
        print(res.get_value(e[j][i]),end="\t")
    print("\n")
print("-------------")

status = optimal, obj. = 33.5
-------------
0.6666666666666666	0	0	1.0	

0.33333333333333337	1.0	0.5	0	

-------------


In [4]:
import datetime as dt
import math
import numpy as np

from minilp import *
import minilp.result

def get_first_non_integral(p, s):
    """ Given a problem p and a result s, returns the first integer variables
    that has a fractional value in s, or None if no such variable was found. """
    for v in p.variables:
        if v.category == int:
            if abs(round(s.get_value(v)) - s.get_value(v)) > 1e-6:
                return v
    return None

def banb(p, lp_solver=None):
    if lp_solver is None:
        lp_solver = solvers.get_default_solver()
    def cmp(l, r):
        if np.isnan(r.objective):
            return True
        if p.sense == min:
            return l.objective < r.objective
        return l.objective > r.objective

    res = minilp.result.result()
    #res = p.lp_solve(lp_solver)
    nodes = [[]] # nodes = list of constraints
    cnt = 0
    
    print('B&B using {} to solve linear relaxation'.format(lp_solver.__class__.__name__))
    
    while nodes:
        if cnt % 10 == 0:
            print('{} {:5d} {:5d} {:9g}'.format(dt.datetime.now().strftime('%T'), cnt, 
                                                len(nodes), res.objective))
        cnt += 1
        cns = nodes.pop()
        cs = p.add_constraints(cns)
        s = p.lp_solve(lp_solver)
        p.del_constraints(cs)
        if s:
            fni = get_first_non_integral(p, s)
            if fni is None: # integral solution
                if cmp(s, res):
                    res = s
                    print('{} {:5d} {:5d} {:9g}*'.format(dt.datetime.now().strftime('%T'), cnt, 
                                                         len(nodes), res.objective))
            elif cmp(s, res):
                fl = math.floor(s.get_value(fni))
                nodes.append(cns + [fni <= fl])
                nodes.append(cns + [fni >= fl + 1])
    return res

In [5]:
result=banb(lp)
print(result)

B&B using docplex to solve linear relaxation
16:58:18     0     1       nan
16:58:19     8     3        31*
16:58:19    10     1        31
16:58:19    20     3        31
status = optimal, obj. = 31.0


In [18]:
from Multiple_knapsack import Multiple_knapsack

pb = Multiple_knapsack("Sources_Files/dc2.in")
a = pb.servers
pb.fileToProblem()
s = pb.flat()
print(s)
a


[7, 8, 8, 8]


[[3, 10],
 [3, 10],
 [2, 5],
 [1, 5],
 [1, 1],
 [4, 50],
 [4, 50],
 [8, 100],
 [8, 100],
 [9, 100]]

In [19]:
c = []
z = []
servers = pb.servers
nb_servers = len(servers)
nb_sacs = len(s)
for server in servers:
    z.append(server[0])
    c.append(server[1])
print(s)

[7, 8, 8, 8]


In [20]:
lp = problem('PLNE')

#existence of a server i in a line j
e = np.array(lp.integer_var_list(nb_sacs*nb_servers,0,1)).reshape(nb_sacs,nb_servers)

#a server appears only one time
for i in range(nb_servers):
    lp.add_constraint(sum(e.T[j]) <= 1)


for j in range(2):
    lp.add_constraint( sum(e[j]*z) <= s[j] )

lp.set_objective( sum(sum(e*c)), sense=max)

In [21]:
result=banb(lp)
print(result)

B&B using docplex to solve linear relaxation
17:04:38     0     1       nan
17:04:38    10     3       nan
17:04:38    11     2      1012*
status = optimal, obj. = 1012.0


In [23]:
e[1][2]

_x12